# 02_entiry-extract

In [1]:
!pip install transformers torch
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.1 MB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.7 MB/s eta 0:00:00a 0:00:01


# install extra library

In [2]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd
import re

## read stored data from DataPlatform

In [3]:
dir_raw_id = 62
dir_process_id = 63

In [4]:
datanode = DataNode()

Init DataNode sucessful!


In [6]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='events_jewelrymagazine.json'))
meta

{'owner': 'nutnicha',
 'human_size': '7.13 KB',
 'name': 'events_jewelrymagazine.json',
 'description': 'file events_jewelrymagazine.json',
 'directory': 62,
 'path': 'nutnicha/DSI310-08/rawData/events_jewelrymagazine.json'}

In [7]:
data = json.load(fp)
data

[{'Title': 'SRK Highlights Sustainability Actions At Gem Genève',
  'Publish_time': 'November 8, 2022',
  'Description': 'Shree Ramkrishna Exports Pvt. Ltd. (SRK), a leading diamond manufacturer, emphasised its sustainability actions at the recently concluded Gem Genève show in Switzerland.',
  'Link': 'https://www.thejewelrymagazine.com/srk-highlights-sustainability-actions-at-gem-geneve/'},
 {'Title': 'Petra Diamonds Halts Mining after Dam Breach',
  'Publish_time': 'November 8, 2022',
  'Description': 'Petra Diamonds has halted operations a wall of the tailings dam at its Williamson mine, in Tanzania, was breached.',
  'Link': 'https://www.thejewelrymagazine.com/petra-diamonds-halts-mining-after-dam-breach/'},
 {'Title': 'MOSE AND INES ARENAS: “JEWELRY CURATORS”',
  'Publish_time': 'August 17, 2018',
  'Description': 'My family and friends call me Mose and I am a "Jewelry Curator". It\'s a job that I inherited from my family, I\'m the third generation in jewelry business.',
  'Link'

# NER (NameEntitiesRegcognition)

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/211 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/980 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [10]:
example = "Five Crystals You Need To Bring In Wealth, Prosperity, and Success Into Your Life"

ner_results = nlp(example)
print(ner_results)

[]


In [11]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [12]:
text = data[5]['Description']
text

'GJEPC in association with Jewellers Development Welfare Association (JDWA) organised an IIJS Visitor Meet on 12th June 2022 in Indore.'

In [13]:
output = nlp(text)
output

[{'entity': 'B-ORG',
  'score': 0.99978954,
  'index': 1,
  'word': '▁G',
  'start': 0,
  'end': 1},
 {'entity': 'I-ORG',
  'score': 0.9996107,
  'index': 2,
  'word': 'JE',
  'start': 1,
  'end': 3},
 {'entity': 'I-ORG',
  'score': 0.99953747,
  'index': 3,
  'word': 'PC',
  'start': 3,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.9998878,
  'index': 7,
  'word': '▁Je',
  'start': 25,
  'end': 28},
 {'entity': 'B-ORG',
  'score': 0.9998005,
  'index': 8,
  'word': 'well',
  'start': 28,
  'end': 32},
 {'entity': 'B-ORG',
  'score': 0.9840365,
  'index': 9,
  'word': 'ers',
  'start': 32,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.9999099,
  'index': 10,
  'word': '▁Development',
  'start': 35,
  'end': 47},
 {'entity': 'I-ORG',
  'score': 0.9999052,
  'index': 11,
  'word': '▁Wel',
  'start': 47,
  'end': 51},
 {'entity': 'I-ORG',
  'score': 0.99986994,
  'index': 12,
  'word': 'fare',
  'start': 51,
  'end': 55},
 {'entity': 'I-ORG',
  'score': 0.99987125,
  'index': 13,
  'wo

In [14]:
post_process(output)

[{'type': 'I-ORG', 'text': ' GJEPC'},
 {'type': 'I-ORG', 'text': ' Jewellers Development Welfare Association'},
 {'type': 'I-ORG', 'text': ' Development Welfare Association'},
 {'type': 'I-ORG', 'text': ' Welfare Association'},
 {'type': 'I-ORG', 'text': ' Association'},
 {'type': 'I-LOC', 'text': ' Indore'}]

# Process events_jewelrymagazine to spo

In [15]:
spo = []
for events_jewelrymagazine in tqdm(data):
    Title = events_jewelrymagazine.get('Title') 
    if Title == None:
        continue
    ner = nlp(Title)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'Title': events_jewelrymagazine.get('Title', ""),
            'Publish_time': events_jewelrymagazine.get('Publish_time', ""),
            'Description': events_jewelrymagazine.get('Description', ""),
            'Link': events_jewelrymagazine.get('Link', "")
        })

  0%|          | 0/17 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame(spo)
df['Title'] = df['Title'].apply(lambda x: x[2:])
df

,Title,Publish_time,Description,Link
0,K Highlights Sustainability Actions At Gem Genève,"November 8, 2022","Shree Ramkrishna Exports Pvt. Ltd. (SRK), a le...",https://www.thejewelrymagazine.com/srk-highlig...
1,K Highlights Sustainability Actions At Gem Genève,"November 8, 2022","Shree Ramkrishna Exports Pvt. Ltd. (SRK), a le...",https://www.thejewelrymagazine.com/srk-highlig...
2,K Highlights Sustainability Actions At Gem Genève,"November 8, 2022","Shree Ramkrishna Exports Pvt. Ltd. (SRK), a le...",https://www.thejewelrymagazine.com/srk-highlig...
3,tra Diamonds Halts Mining after Dam Breach,"November 8, 2022",Petra Diamonds has halted operations a wall of...,https://www.thejewelrymagazine.com/petra-diamo...
4,tra Diamonds Halts Mining after Dam Breach,"November 8, 2022",Petra Diamonds has halted operations a wall of...,https://www.thejewelrymagazine.com/petra-diamo...
5,SE AND INES ARENAS: “JEWELRY CURATORS”,"August 17, 2018",My family and friends call me Mose and I am a ...,https://www.thejewelrymagazine.com/mose-and-in...
6,SE AND INES ARENAS: “JEWELRY CURATORS”,"August 17, 2018",My family and friends call me Mose and I am a ...,https://www.thejewelrymagazine.com/mose-and-in...
7,SE AND INES ARENAS: “JEWELRY CURATORS”,"August 17, 2018",My family and friends call me Mose and I am a ...,https://www.thejewelrymagazine.com/mose-and-in...
8,SE AND INES ARENAS: “JEWELRY CURATORS”,"August 17, 2018",My family and friends call me Mose and I am a ...,https://www.thejewelrymagazine.com/mose-and-in...
9,"mfields Sells Huge 187,775-ct Emerald Cluster","November 4, 2022","A colossal emerald cluster, weighing 187,775 c...",https://www.thejewelrymagazine.com/gemfields-s...


In [17]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, 
                                                                                                          name='events_jewelrymagazine.json')])

{'sucess': True,
 'file_id': 744,
 'path': 'nutnicha/DSI310-08/processData/spo.parquet',
 'lineage': True}